In [3]:
import pandas as pd
import os
from tqdm import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


This NB just works of an already extracted zip. the directory should be the unzipped directory of that zip.

## Get data

### Load csvs

In [11]:
directory = '2023-citibike-tripdata/'
data = pd.DataFrame()
for foldername, subfolders, filenames in os.walk(directory):
    for file in tqdm(os.listdir(foldername)):
        filename = os.fsdecode(file)
        if filename.endswith('.csv'):
            month_df = pd.read_csv(os.path.join(foldername, filename), low_memory=False)
            data = pd.concat([data, month_df])

100%|██████████| 3/3 [00:35<00:00, 11.67s/it]


In [13]:
data.shape

(35107030, 13)

In [16]:
# there are about 300 rides with dock name starting with 'id'
# remove these so columns can be floats

data['start_station_id'] = pd.to_numeric(data['start_station_id'], errors='coerce')
data['end_station_id'] = pd.to_numeric(data['end_station_id'], errors='coerce')
data = data.dropna()

In [18]:
data.to_parquet('2023-citibike-tripdata/2023_data.parquet')

In [14]:
# we need to filter out self-loops which are quite common
filtered_data = data[data['start_station_id'] != data['end_station_id']]
unique_trips = filtered_data.groupby(['start_station_id', 'end_station_id']).size().reset_index(name='trip_count')

In [19]:
# join back the lats and longs for each station
# the lats/long for station ids differ slightly for each trip, so drop dupes and keep the first since they are all very close

starts = data[['start_station_id', 'start_lat', 'start_lng']].drop_duplicates(subset='start_station_id', keep='first')
ends = data[['end_station_id', 'end_lat', 'end_lng']].drop_duplicates(subset='end_station_id', keep='first')
unique_trips = unique_trips.merge(starts, on='start_station_id', how='left')
unique_trips = unique_trips.merge(ends, on='end_station_id', how='left')

In [20]:
unique_trips.to_parquet('2023-citibike-tripdata/unique_trips.parquet')